# Creación de Dataset de Pares Pregunta-Respuesta
En este cuaderno se muestra cómo generar un conjunto de datos de pares de pregunta-respuesta para entrenar y probar un modelo de lenguaje. Se utilizan diversas plantillas en lenguaje natural que se combinan con valores extraídos de una base de datos (BBDD) para lograr una gran variedad de consultas.

A continuación, se detallan los pasos:
1. **Conexión a la base de datos**.
2. **Creación de las plantillas en lenguaje natural (NL)** y las consultas SQL correspondientes.
3. **Generación masiva del dataset**: se combinan las plantillas con valores reales de la base de datos para componer preguntas y sus correspondientes consultas SQL.
4. **Creación de subconjuntos (Training, Test, Validation)**: se muestra cómo se pueden extraer muestras estratificadas del dataset.
5. **Validación de las queries**: se comprueba que las consultas generadas se ejecutan sin errores.


## 1. Conectar con BBDD
En esta sección se realiza la conexión con la base de datos PostgreSQL para extraer valores que luego se utilizarán al crear las plantillas.


In [5]:
from langchain_community.utilities import SQLDatabase

usuario = 'postgres'
password = 'place_rag_password'
host = 'localhost'     # o la IP/URL de tu servidor
puerto = '5432'        # puerto por defecto de PostgreSQL
base_datos = 'place_rag_db'

# Crear la URL de conexión
uri = f"postgresql+psycopg2://{usuario}:{password}@{host}:{puerto}/{base_datos}"

db = SQLDatabase.from_uri(uri)

# Verificaciones básicas
print(db.dialect)                  # Para verificar el tipo de base de datos
print(db.get_usable_table_names()) # Para listar las tablas de la BD
db.run("SELECT * FROM documentos LIMIT 30;")  # Ejemplo de consulta

postgresql
['documentos', 'entidades', 'expedientes', 'paises', 'regiones']


"[('PCAP 50 equipos trabajo en movilidad Anexo I acuerdo MP.pdf', 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=0b72a403-f89f-44fe-852d-dcba825cdef4', 'Pliego Administrativo', '2023/20'), ('PPT Suministro 50 portatiles Anexo II acuerdo MP.pdf', 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=91b4b1e6-5e64-46e7-8091-27febb59c5fd', 'Pliego Técnico', '2023/20'), ('04 PCAP 35 2023 FIRMADO.pdf', 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=8bed1139-4bab-4327-9849-9d3f9ed091f3', 'Pliego Administrativo', 'CON 35/2023 SE AB'), ('03 PPT GRUPOS DE BOMBEO 2023 FIRMADA.pdf', 'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&sour

## 2. Creación de plantillas de lenguaje natural (NL) y consultas SQL
A continuación se definen:
- **Plantillas en lenguaje natural**: Varias frases para cada tipo de petición (por región, tipo de expediente, cuantía, etc.).
- **Consultas SQL**: La plantilla de la consulta que luego se formateará con los valores apropiados.


In [9]:
import json

with open("plantillas_nl.json", "r", encoding="utf-8") as file:
    plantillas_nl = json.load(file)
    
plantillas_nl = plantillas_nl["plantillas_nl"]

In [10]:
# Casos de uso
# 1. Consultar expedientes por región
# 2. Consultar expedientes por tipo
# 3. Consultar expedientes por entidad
# 4. Consultar expedientes por estado
# 5. Consultar expedientes por cuantía
# 6. Combinaciones de las anteriores

def encontrar_strings_repetidos(diccionario):
    """
    Recorre un diccionario donde cada valor es una lista de strings y 
    encuentra todos los strings que se repiten dentro de cada lista.
    """
    repetidos = set()

    for clave, lista_strings in diccionario.items():
        if not isinstance(lista_strings, list):
            print(f"Advertencia: El valor de la clave '{clave}' no es una lista. Se omitirá.")
            continue

        vistos = set()
        for string in lista_strings:
            if string in vistos:
                repetidos.add(string)
            else:
                vistos.add(string)
    return list(repetidos)

strings_repetidos = encontrar_strings_repetidos(plantillas_nl)
print("Strings repetidos encontrados:", strings_repetidos)

# Definir las plantillas SQL
plantillas_sql = {
    # ----- CONSULTAS SIMPLES -----
    'region': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}';"
    ),
    'entidad': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}';"
    ),
    'tipo': (
        "SELECT * "
        "FROM expedientes "
        "WHERE procurement_project_type_code = '{tipo}';"
    ),
    'estado': (
        "SELECT * "
        "FROM expedientes "
        "WHERE contract_folder_status = '{estado}';"
    ),
    'cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "WHERE total_amount < {cuantia};"
    ),
    'cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "WHERE total_amount > {cuantia};"
    ),
    
    # ----- CONSULTAS COMBINADAS -----
    # Combinaciones con 'region'
    'region_tipo': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.procurement_project_type_code = '{tipo}';"
    ),
    'region_estado': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.contract_folder_status = '{estado}';"
    ),
    'region_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.total_amount < {cuantia};"
    ),
    'region_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.total_amount > {cuantia};"
    ),
    'region_tipo_estado': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.contract_folder_status = '{estado}';"
    ),
    'region_tipo_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.total_amount < {cuantia};"
    ),
    'region_tipo_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.total_amount > {cuantia};"
    ),
    'region_estado_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.contract_folder_status = '{estado}' "
        "AND expedientes.total_amount < {cuantia};"
    ),
    'region_estado_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.contract_folder_status = '{estado}' "
        "AND expedientes.total_amount > {cuantia};"
    ),
    'region_tipo_estado_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.contract_folder_status = '{estado}' "
        "AND expedientes.total_amount < {cuantia};"
    ),
    'region_tipo_estado_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "JOIN regiones ON entidades.party_subentity_code = regiones.country_subentity_code "
        "WHERE regiones.country_subentity_name = '{region}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.contract_folder_status = '{estado}' "
        "AND expedientes.total_amount > {cuantia};"
    ),
    
    # Combinaciones con 'entidad'
    'entidad_tipo': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.procurement_project_type_code = '{tipo}';"
    ),
    'entidad_estado': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.contract_folder_status = '{estado}';"
    ),
    'entidad_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.total_amount < {cuantia};"
    ),
    'entidad_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.total_amount > {cuantia};"
    ),
    'entidad_tipo_estado': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.contract_folder_status = '{estado}';"
    ),
    'entidad_tipo_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.total_amount < {cuantia};"
    ),
    'entidad_tipo_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.total_amount > {cuantia};"
    ),
    'entidad_estado_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.contract_folder_status = '{estado}' "
        "AND expedientes.total_amount < {cuantia};"
    ),
    'entidad_estado_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.contract_folder_status = '{estado}' "
        "AND expedientes.total_amount > {cuantia};"
    ),
    'entidad_tipo_estado_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.contract_folder_status = '{estado}' "
        "AND expedientes.total_amount < {cuantia};"
    ),
    'entidad_tipo_estado_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "JOIN entidades ON expedientes.party_nif = entidades.nif "
        "WHERE entidades.name = '{entidad}' "
        "AND expedientes.procurement_project_type_code = '{tipo}' "
        "AND expedientes.contract_folder_status = '{estado}' "
        "AND expedientes.total_amount > {cuantia};"
    ),
    
    # Combinaciones con 'tipo'
    'tipo_estado': (
        "SELECT * "
        "FROM expedientes "
        "WHERE procurement_project_type_code = '{tipo}' "
        "AND contract_folder_status = '{estado}';"
    ),
    'tipo_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "WHERE procurement_project_type_code = '{tipo}' "
        "AND total_amount < {cuantia};"
    ),
    'tipo_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "WHERE procurement_project_type_code = '{tipo}' "
        "AND total_amount > {cuantia};"
    ),
    'tipo_estado_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "WHERE procurement_project_type_code = '{tipo}' "
        "AND contract_folder_status = '{estado}' "
        "AND total_amount < {cuantia};"
    ),
    'tipo_estado_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "WHERE procurement_project_type_code = '{tipo}' "
        "AND contract_folder_status = '{estado}' "
        "AND total_amount > {cuantia};"
    ),
    
    # Combinaciones con 'estado'
    'estado_cuantia_inferior': (
        "SELECT * "
        "FROM expedientes "
        "WHERE contract_folder_status = '{estado}' "
        "AND total_amount < {cuantia};"
    ),
    'estado_cuantia_superior': (
        "SELECT * "
        "FROM expedientes "
        "WHERE contract_folder_status = '{estado}' "
        "AND total_amount > {cuantia};"
    )  
}


Strings repetidos encontrados: []


## 3. Generación masiva del dataset
En esta sección se generan preguntas reales sustituyendo los placeholders (por ejemplo, `{region}`, `{tipo}`, etc.) con valores extraídos de la base de datos. También se crea la consulta SQL correspondiente.
La salida final es un `DataFrame` de Pandas que contiene:
- La **pregunta** en lenguaje natural.
- La **consulta** SQL.
- La **tabla** principal involucrada.
- Los **valores** usados para cada placeholder.
- La **categoría** (tipo de intención o consulta).


In [18]:
import ast
import re
import random
import pandas as pd

def query_as_list(db, query):
    res = db.run(query)
    
    # Verifica si la respuesta contiene 'Decimal('
    if "Decimal(" in res:
        # Extrae todos los números dentro de Decimal('...')
        numbers = re.findall(r"Decimal\('([\\d.]+)'\)", res)
        # Convierte las cadenas extraídas a float (o a Decimal, según requiera)
        processed = [float(num) for num in numbers]
    else:
        try:
            # Intenta evaluar la respuesta como lista de tuplas
            parsed = ast.literal_eval(res)
            # Aplanar la lista de tuplas, si fuese necesario
            flat = [el[0] for el in parsed if el]
            # Limpieza con regex en caso de que sean strings
            processed = [re.sub(r"\\b\\d+\\b", "", string).strip() for string in flat]
        except Exception as e:
            raise ValueError(f"Error al evaluar la respuesta de la base de datos: {e}")
    
    # Elimina duplicados
    return list(set(processed))

# Se asume que estas tablas y valores existen en la BD
regiones = query_as_list(db,"SELECT country_subentity_name FROM regiones WHERE country_code = 'ES';")
entidades = query_as_list(db,"SELECT name FROM entidades;")
tipos = query_as_list(db,"SELECT procurement_project_type_code FROM expedientes;")
estados = query_as_list(db,"SELECT contract_folder_status FROM expedientes;")

# Generamos varias cuantías aleatorias
cuantias = [(random.randint(10000, 1000000) // 1000) * 1000 for _ in range(100)]

# Función para extraer el nombre de la tabla desde la consulta SQL
def extraer_tabla(sql):
    match = re.search(r'FROM\\s+(\\w+)', sql, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return 'Desconocida'

# Función auxiliar para seleccionar un valor sin apostrofe
def seleccionar_valor_sin_apostrofe(lista_valores, max_intentos=10):
    for _ in range(max_intentos):
        valor = random.choice(lista_valores)
        if isinstance(valor, str) and "'" in valor:
            continue
        return valor
    # Si no se encuentra valor válido
    return 'Desconocido'

# Mapeo de placeholders a sus listas
placeholder_map = {
    'region': regiones,
    'entidad': entidades,
    'tipo': tipos,
    'estado': estados,
    'cuantia': cuantias
}

filas = []  # aquí almacenaremos los registros

for clave, plantillas_pregunta in plantillas_nl.items():
    plantilla_sql = plantillas_sql.get(clave)
    
    # Si no existe una consulta SQL para la clave, pasar
    if not plantilla_sql:
        continue
    
    # Iterar sobre cada plantilla de pregunta
    for pregunta_template in plantillas_pregunta:
        # Generar 5 variaciones por plantilla
        for _ in range(5):
            placeholders = re.findall(r'\{(\w+)\}', pregunta_template)
            valores = {}
            for placeholder in placeholders:
                lista_valores = placeholder_map.get(placeholder, ['Desconocido'])
                
                if placeholder == 'cuantia':
                    valores[placeholder] = random.choice(lista_valores)
                else:
                    valor_seleccionado = seleccionar_valor_sin_apostrofe(lista_valores)
                    valores[placeholder] = valor_seleccionado

            # Copiamos para la pregunta (por si queremos formatear distinto)
            valores_pregunta = valores.copy()
            if 'cuantia' in valores_pregunta:
                valores_pregunta['cuantia'] = f"{valores_pregunta['cuantia']}€"

            pregunta = pregunta_template.format(**valores_pregunta)
            consulta = plantilla_sql.format(**valores)

            # Extraer nombre de la tabla
            tabla = extraer_tabla(consulta)

            filas.append({
                'Pregunta': pregunta,
                'Consulta': consulta,
                'Tabla': tabla,
                'Valores': valores,
                'Categoría': clave
            })

# Crear el DataFrame
df_place_dataset = pd.DataFrame(filas, columns=['Pregunta', 'Consulta', 'Tabla', 'Valores', 'Categoría'])
df_place_dataset


,Pregunta,Consulta,Tabla,Valores,Categoría
0,¿Cuáles son los expedientes de Teruel?,SELECT * FROM expedientes JOIN entidades ON ex...,Desconocida,{'region': 'Teruel'},region
1,¿Cuáles son los expedientes de Málaga?,SELECT * FROM expedientes JOIN entidades ON ex...,Desconocida,{'region': 'Málaga'},region
2,¿Cuáles son los expedientes de Eivissa y Forme...,SELECT * FROM expedientes JOIN entidades ON ex...,Desconocida,{'region': 'Eivissa y Formentera'},region
3,¿Cuáles son los expedientes de Salamanca?,SELECT * FROM expedientes JOIN entidades ON ex...,Desconocida,{'region': 'Salamanca'},region
4,¿Cuáles son los expedientes de Aragón?,SELECT * FROM expedientes JOIN entidades ON ex...,Desconocida,{'region': 'Aragón'},region
...,...,...,...,...,...
12245,Quiero ver las licitaciones Publicado más reci...,SELECT * FROM expedientes WHERE contract_folde...,Desconocida,"{'estado': 'Publicado', 'cuantia': 790000}",estado_cuantia_superior
12246,Quiero ver las licitaciones En evaluación más ...,SELECT * FROM expedientes WHERE contract_folde...,Desconocida,"{'estado': 'En evaluación', 'cuantia': 949000}",estado_cuantia_superior
12247,Quiero ver las licitaciones Publicado más reci...,SELECT * FROM expedientes WHERE contract_folde...,Desconocida,"{'estado': 'Publicado', 'cuantia': 343000}",estado_cuantia_superior
12248,Quiero ver las licitaciones Anuncio Previo más...,SELECT * FROM expedientes WHERE contract_folde...,Desconocida,"{'estado': 'Anuncio Previo', 'cuantia': 342000}",estado_cuantia_superior


Finalmente, exportamos el DataFrame a CSV para su posterior uso:


In [ ]:
df_place_dataset.to_csv("place_dataset.csv")
df_place_dataset.head(10)  # Mostrar las primeras filas como ejemplo

## 4. Creación de subconjuntos (Test, Validación, Entrenamiento)
En esta sección se muestra un ejemplo de cómo se puede hacer un muestreo estratificado para construir subconjuntos de datos. La idea es mantener la proporción original de categorías en cada subconjunto.

In [8]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Cargamos el dataset generado
df = pd.read_csv('datasets/place_dataset.csv')

print("Número total de filas originales:", len(df))
print("Distribución original de categorías:")
print(df['Categoría'].value_counts())

# Definimos un tamaño de muestra
sample_size = 2400
# Calculamos la distribución de categorías en el dataset
category_counts = df['Categoría'].value_counts()
total_rows = category_counts.sum()
category_proportions = category_counts / total_rows
samples_per_category = (category_proportions * sample_size).round().astype(int)

print("\nNúmero de muestras por categoría:")
print(samples_per_category)

# Muestreo estratificado
grouped = df.groupby('Categoría')
samples = []
for category, count in samples_per_category.items():
    group = grouped.get_group(category)
    if len(group) < count:
        raise ValueError(f"No hay suficientes filas en la categoría '{category}' para muestrear {count} filas.")
    sampled_group = group.sample(n=count, random_state=55)
    samples.append(sampled_group)

sampled_df = pd.concat(samples)
sampled_df = sampled_df.drop('Unnamed: 0', axis=1, errors='ignore')

print("Número total de filas de la muestra:", len(sampled_df))
print("\nDistribución de categorías en la muestra:")
print(sampled_df['Categoría'].value_counts())

sampled_df.to_csv('datasets/sampled_place_dataset_2.csv', index=False)


Número total de filas originales: 12250
Distribución original de categorías:
Categoría
region                                  350
entidad_tipo_estado_cuantia_inferior    350
entidad_cuantia_superior                350
entidad_tipo_estado                     350
entidad_tipo_cuantia_inferior           350
entidad_tipo_cuantia_superior           350
entidad_estado_cuantia_inferior         350
entidad_estado_cuantia_superior         350
entidad_tipo_estado_cuantia_superior    350
entidad_estado                          350
tipo_estado                             350
tipo_cuantia_inferior                   350
tipo_cuantia_superior                   350
tipo_estado_cuantia_inferior            350
tipo_estado_cuantia_superior            350
estado_cuantia_inferior                 350
entidad_cuantia_inferior                350
entidad_tipo                            350
entidad                                 350
region_cuantia_inferior                 350
tipo                             

### Análisis básico
Podemos contar la frecuencia de comandos SQL en las consultas generadas:


In [3]:
from collections import Counter
import re

# Definir una lista de comandos SQL a analizar
sql_commands = ['SELECT', 'FROM', 'WHERE', 'JOIN', 'ON', 'GROUP BY', 'ORDER BY', 'LIMIT', 'INSERT', 'UPDATE', 'DELETE']

# Inicializar un contador
sql_counter = Counter()

# Compilar patrones regex para cada comando, respetando mayúsculas y delimitando palabras
patterns = {comando: re.compile(r'\b' + re.escape(comando) + r'\b') for comando in sql_commands}

# Función para contar comandos en una consulta SQL
def count_sql_commands(sql_query):
    for comando, pattern in patterns.items():
        # Encontrar todas las coincidencias del comando
        matches = pattern.findall(sql_query)
        count = len(matches)
        if count > 0:
            sql_counter[comando] += count

# Aplicar la función a la columna 'Consulta'
sampled_df['Consulta'].apply(count_sql_commands)

# Mostrar los resultados
print("\nFrecuencia de comandos SQL en las consultas:")
for comando, count in sql_counter.items():
    print(f"{comando}: {count}")



Frecuencia de comandos SQL en las consultas:
SELECT: 2415
FROM: 2415
WHERE: 2415
JOIN: 2484
ON: 2484


## **5. Validación de las queries generadas**
Por último, podemos ejecutar las consultas en la base de datos para comprobar que no dan error.


In [6]:
from sqlalchemy.exc import SQLAlchemyError

def ejecutar_consultas(df, db):
    resultados = []

    for idx, row in df.iterrows():
        consulta = row['Consulta']
        try:
            db.run(consulta)
        except SQLAlchemyError as e:
            error_msg = str(e.__dict__['orig']) if e.__dict__.get('orig') else str(e)
            resultados.append([idx, error_msg])
    return resultados

errores = ejecutar_consultas(sampled_df, db)
errores


[]

Si la lista `errores` se encuentra vacía (`[]`), significa que todas las consultas generadas han sido válidas en la base de datos.

Con esto concluimos la generación y validación del dataset de pares pregunta-respuesta.